# TV Script Generation
In this project, you'll generate your own [Simpsons](https://en.wikipedia.org/wiki/The_Simpsons) TV scripts using RNNs.  You'll be using part of the [Simpsons dataset](https://www.kaggle.com/wcukierski/the-simpsons-by-the-data) of scripts from 27 seasons.  The Neural Network you'll build will generate a new TV script for a scene at [Moe's Tavern](https://simpsonswiki.com/wiki/Moe's_Tavern).
## Get the Data
The data is already provided for you.  You'll be using a subset of the original dataset.  It consists of only the scenes in Moe's Tavern.  This doesn't include other versions of the tavern, like "Moe's Cavern", "Flaming Moe's", "Uncle Moe's Family Feed-Bag", etc..

In [1]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import helper

data_dir = './data/simpsons/moes_tavern_lines.txt'
text = helper.load_data(data_dir)
# Ignore notice, since we don't use it for analysing the data
text = text[81:]

## Explore the Data
Play around with `view_sentence_range` to view different parts of the data.

In [2]:
view_sentence_range = (0, 10)

"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import numpy as np

print('Dataset Stats')
print('Roughly the number of unique words: {}'.format(len({word: None for word in text.split()})))
scenes = text.split('\n\n')
print('Number of scenes: {}'.format(len(scenes)))
sentence_count_scene = [scene.count('\n') for scene in scenes]
print('Average number of sentences in each scene: {}'.format(np.average(sentence_count_scene)))

sentences = [sentence for scene in scenes for sentence in scene.split('\n')]
print('Number of lines: {}'.format(len(sentences)))
word_count_sentence = [len(sentence.split()) for sentence in sentences]
print('Average number of words in each line: {}'.format(np.average(word_count_sentence)))

print()
print('The sentences {} to {}:'.format(*view_sentence_range))
print('\n'.join(text.split('\n')[view_sentence_range[0]:view_sentence_range[1]]))

Dataset Stats
Roughly the number of unique words: 11492
Number of scenes: 262
Average number of sentences in each scene: 15.251908396946565
Number of lines: 4258
Average number of words in each line: 11.50164396430249

The sentences 0 to 10:

Moe_Szyslak: (INTO PHONE) Moe's Tavern. Where the elite meet to drink.
Bart_Simpson: Eh, yeah, hello, is Mike there? Last name, Rotch.
Moe_Szyslak: (INTO PHONE) Hold on, I'll check. (TO BARFLIES) Mike Rotch. Mike Rotch. Hey, has anybody seen Mike Rotch, lately?
Moe_Szyslak: (INTO PHONE) Listen you little puke. One of these days I'm gonna catch you, and I'm gonna carve my name on your back with an ice pick.
Moe_Szyslak: What's the matter Homer? You're not your normal effervescent self.
Homer_Simpson: I got my problems, Moe. Give me another one.
Moe_Szyslak: Homer, hey, you should not drink to forget your problems.
Barney_Gumble: Yeah, you should only drink to enhance your social skills.



## Implement Preprocessing Functions
The first thing to do to any dataset is preprocessing.  Implement the following preprocessing functions below:
- Lookup Table
- Tokenize Punctuation

### Lookup Table
To create a word embedding, you first need to transform the words to ids.  In this function, create two dictionaries:
- Dictionary to go from the words to an id, we'll call `vocab_to_int`
- Dictionary to go from the id to word, we'll call `int_to_vocab`

Return these dictionaries in the following tuple `(vocab_to_int, int_to_vocab)`

In [3]:
import numpy as np
import problem_unittests as tests

# Using Counter for word counts
from collections import Counter

def create_lookup_tables(text):
    """
    Create lookup tables for vocabulary
    :param text: The text of tv scripts split into words
    :return: A tuple of dicts (vocab_to_int, int_to_vocab)
    """
    # DONE: Implement Function
    word_counts = Counter(text)
    sorted_vocab = sorted(word_counts, key=word_counts.get, reverse=True)
    int_to_vocab = {ii: word for ii, word in enumerate(sorted_vocab)}
    vocab_to_int = dict(zip(int_to_vocab.values(), int_to_vocab.keys()))
    return vocab_to_int, int_to_vocab


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_create_lookup_tables(create_lookup_tables)

Tests Passed


### Tokenize Punctuation
We'll be splitting the script into a word array using spaces as delimiters.  However, punctuations like periods and exclamation marks make it hard for the neural network to distinguish between the word "bye" and "bye!".

Implement the function `token_lookup` to return a dict that will be used to tokenize symbols like "!" into "||Exclamation_Mark||".  Create a dictionary for the following symbols where the symbol is the key and value is the token:
- Period ( . )
- Comma ( , )
- Quotation Mark ( " )
- Semicolon ( ; )
- Exclamation mark ( ! )
- Question mark ( ? )
- Left Parentheses ( ( )
- Right Parentheses ( ) )
- Dash ( -- )
- Return ( \n )

This dictionary will be used to token the symbols and add the delimiter (space) around it.  This separates the symbols as it's own word, making it easier for the neural network to predict on the next word. Make sure you don't use a token that could be confused as a word. Instead of using the token "dash", try using something like "||dash||".

In [4]:
def token_lookup():
    """
    Generate a dict to turn punctuation into a token.
    :return: Tokenize dictionary where the key is the punctuation and the value is the token
    """
    # DONE: Implement Function
    lookup = {
        '.': '||PERIOD||',
        ',': '||COMMA||',
        '"': '||QUOTATION_MARK||',
        ';': '||SEMICOLON||',
        #':': '||COLON||',
        '!': '||EXCLAMATION_MARK||',
        '?': '||QUESTION_MARK||',
        '(': '||LEFT_PAREN||',
        ')': '||RIGHT_PAREN||',
        '--': '||DASH||',
        '\n': '||RETURN||'
    }
    return lookup

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_tokenize(token_lookup)

Tests Passed


## Preprocess all the data and save it
Running the code cell below will preprocess all the data and save it to file.

In [5]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
# Preprocess Training, Validation, and Testing Data
helper.preprocess_and_save_data(data_dir, token_lookup, create_lookup_tables)

 ||return|| moe_szyslak:  ||left_paren|| into phone ||right_paren||  moe's tavern ||period||  where 
['||return||', 'moe_szyslak:', '||left_paren||', 'into', 'phone', '||right_paren||', "moe's", 'tavern', '||period||', 'where', 'the', 'elite', 'meet', 'to', 'drink', '||period||', '||return||', 'bart_simpson:', 'eh', '||comma||', 'yeah', '||comma||', 'hello', '||comma||', 'is', 'mike', 'there', '||question_mark||', 'last', 'name', '||comma||', 'rotch', '||period||', '||return||', 'moe_szyslak:', '||left_paren||', 'into', 'phone', '||right_paren||', 'hold', 'on', '||comma||', "i'll", 'check', '||period||', '||left_paren||', 'to', 'barflies', '||right_paren||', 'mike', 'rotch', '||period||', 'mike', 'rotch', '||period||', 'hey', '||comma||', 'has', 'anybody', 'seen', 'mike', 'rotch', '||comma||', 'lately', '||question_mark||', '||return||', 'moe_szyslak:', '||left_paren||', 'into', 'phone', '||right_paren||', 'listen', 'you', 'little', 'puke', '||period||', 'one', 'of', 'these', 'days', "

# Check Point
This is your first checkpoint. If you ever decide to come back to this notebook or have to restart the notebook, you can start from here. The preprocessed data has been saved to disk.

In [6]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import helper
import numpy as np
import problem_unittests as tests

int_text, vocab_to_int, int_to_vocab, token_dict = helper.load_preprocess()

## Build the Neural Network
You'll build the components necessary to build a RNN by implementing the following functions below:
- get_inputs
- get_init_cell
- get_embed
- build_rnn
- build_nn
- get_batches

### Check the Version of TensorFlow and Access to GPU

In [7]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
from distutils.version import LooseVersion
import warnings
import tensorflow as tf

# Check TensorFlow Version
assert LooseVersion(tf.__version__) >= LooseVersion('1.0'), 'Please use TensorFlow version 1.0 or newer'
print('TensorFlow Version: {}'.format(tf.__version__))

# Check for a GPU
if not tf.test.gpu_device_name():
    warnings.warn('No GPU found. Please use a GPU to train your neural network.')
else:
    print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))

TensorFlow Version: 1.0.1
Default GPU Device: /gpu:0


### Input
Implement the `get_inputs()` function to create TF Placeholders for the Neural Network.  It should create the following placeholders:
- Input text placeholder named "input" using the [TF Placeholder](https://www.tensorflow.org/api_docs/python/tf/placeholder) `name` parameter.
- Targets placeholder
- Learning Rate placeholder

Return the placeholders in the following tuple `(Input, Targets, LearningRate)`

In [8]:
def get_inputs():
    """
    Create TF Placeholders for input, targets, and learning rate.
    :return: Tuple (input, targets, learning rate)
    """
    # DONE: Implement Function
    
    # Declare placeholders we'll feed into the graph
    Input = tf.placeholder(tf.int32, [None, None], name='input')
    Targets = tf.placeholder(tf.int32, [None, None], name='targets')
    
    # Declaring Learning Rate placeholder
    LearningRate = tf.placeholder(tf.float32, name='learning_rate')
    
    #Return the placeholders in the following tuple (Input, Targets, LearningRate)
    return Input, Targets, LearningRate


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_get_inputs(get_inputs)

Tests Passed


### Build RNN Cell and Initialize
Stack one or more [`BasicLSTMCells`](https://www.tensorflow.org/api_docs/python/tf/contrib/rnn/BasicLSTMCell) in a [`MultiRNNCell`](https://www.tensorflow.org/api_docs/python/tf/contrib/rnn/MultiRNNCell).
- The Rnn size should be set using `rnn_size`
- Initalize Cell State using the MultiRNNCell's [`zero_state()`](https://www.tensorflow.org/api_docs/python/tf/contrib/rnn/MultiRNNCell#zero_state) function
    - Apply the name "initial_state" to the initial state using [`tf.identity()`](https://www.tensorflow.org/api_docs/python/tf/identity)

Return the cell and initial state in the following tuple `(Cell, InitialState)`

In [9]:
## Added 2 more parameters to function signature. Why is 'num_layers' not in the original signature?
# num_layers - Stack one or more BasicLSTMCells in a MultiRNNCell.
# keep_prob - Keep Prob parameter for the LSTM Cells

def get_init_cell(batch_size, rnn_size, num_layers = 2, keep_prob = 1.0):
    """
    Create an RNN Cell and initialize it.
    :param batch_size: Size of batches
    :param rnn_size: Size of RNNs
    :return: Tuple (cell, initialize state)
    """

    # DONE: Implement Function
    def build_cell(lstm_size, keep_prob):
        # Use a basic LSTM cell
        lstm = tf.contrib.rnn.BasicLSTMCell(lstm_size)
        
        # Add dropout to the cell if needed
        if keep_prob < 1.0:
            lstm = tf.contrib.rnn.DropoutWrapper(lstm, output_keep_prob=keep_prob)
        
        return lstm
    
    
    # Stack up multiple LSTM layers, for deep learning
    cell = tf.contrib.rnn.MultiRNNCell([build_cell(rnn_size, keep_prob) for _ in range(num_layers)])
    
    # Apply initial state
    initial_state = tf.identity(cell.zero_state(batch_size, tf.float32), name='initial_state')
    
    return cell, initial_state

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_get_init_cell(get_init_cell)

Tests Passed


### Word Embedding
Apply embedding to `input_data` using TensorFlow.  Return the embedded sequence.

In [16]:
def get_embed(input_data, vocab_size, embed_dim):
    """
    Create embedding for <input_data>.
    :param input_data: TF placeholder for text input.
    :param vocab_size: Number of words in vocabulary.
    :param embed_dim: Number of embedding dimensions
    :return: Embedded input.
    """
    # DONE: Implement Function
    with tf.device('/cpu:0'):
        W = tf.Variable(tf.random_uniform([vocab_size, embed_dim], -1.0, 1.0))
        embedded_input = tf.nn.embedding_lookup(W, input_data)

    #print('input_data', input_data)
    #print('vocab_size', vocab_size)
    #print('embed_dim',embed_dim)
    #print('embedded_input.shape', embedded_input.shape)
    
    return embedded_input
    
"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_get_embed(get_embed)

Tests Passed


### Build RNN
You created a RNN Cell in the `get_init_cell()` function.  Time to use the cell to create a RNN.
- Build the RNN using the [`tf.nn.dynamic_rnn()`](https://www.tensorflow.org/api_docs/python/tf/nn/dynamic_rnn)
 - Apply the name "final_state" to the final state using [`tf.identity()`](https://www.tensorflow.org/api_docs/python/tf/identity)

Return the outputs and final_state state in the following tuple `(Outputs, FinalState)` 

In [19]:
def build_rnn(cell, inputs, initial_state=None, dtype=tf.float32):
    """
    Create a RNN using a RNN Cell
    :param cell: RNN Cell
    :param inputs: Input text data
    :return: Tuple (Outputs, Final State)
    """
    # DONE: Implement Function
    print('cell', cell)
    print('inputs', inputs)
    
    # Run each sequence step through the RNN and collect the outputs
    outputs, state = tf.nn.dynamic_rnn(cell, inputs, initial_state=initial_state, dtype=dtype)
    print('outputs', outputs)
    print('state',state)
    state = tf.identity(state, name='final_state')
    
    return outputs, state


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_build_rnn(build_rnn)

cell <tensorflow.contrib.rnn.python.ops.core_rnn_cell_impl.MultiRNNCell object at 0x00000203F91C3E10>
inputs Tensor("Placeholder:0", shape=(?, ?, 256), dtype=float32)
outputs Tensor("rnn/transpose:0", shape=(?, ?, 256), dtype=float32)
state (LSTMStateTuple(c=<tf.Tensor 'rnn/while/Exit_2:0' shape=(?, 256) dtype=float32>, h=<tf.Tensor 'rnn/while/Exit_3:0' shape=(?, 256) dtype=float32>), LSTMStateTuple(c=<tf.Tensor 'rnn/while/Exit_4:0' shape=(?, 256) dtype=float32>, h=<tf.Tensor 'rnn/while/Exit_5:0' shape=(?, 256) dtype=float32>))
Tests Passed


### Build outputs

In [42]:
def build_output(lstm_output, in_size, out_size):
    ''' Build a fully connected layer, return linear activation output and logits.
    
        Arguments
        ---------
        
        x: Input tensor
        in_size: Size of the input tensor, for example, size of the LSTM cells
        out_size: Size of this linear layer
    
    '''

    # Reshape output so it's a bunch of rows, one row for each step for each sequence.
    # That is, the shape should be batch_size*num_steps rows by lstm_size columns
    
    seq_output = tf.concat(lstm_output, axis=1)
    x = tf.reshape(seq_output, [-1, in_size])
    
    print("seq_output",seq_output)
    print("x",x)

    
    # Connect the RNN outputs to a softmax layer
    w = tf.Variable(tf.truncated_normal((in_size, out_size), stddev=0.1))
    b = tf.Variable(tf.zeros(out_size))

    # Since output is a bunch of rows of RNN cell outputs, logits will be a bunch
    # of rows of logit outputs, one for each step and sequence
    logits = tf.matmul(x, w) + b
    print('logits.shape', logits.shape)
    logits = tf.reshape(logits, [lstm., 5, out_size] )
    
    # Use softmax to get the probabilities for predicted characters
    #out = tf.nn.softmax(logits, name='predictions')
    
    return logits

### Build the Neural Network
Apply the functions you implemented above to:
- Apply embedding to `input_data` using your `get_embed(input_data, vocab_size, embed_dim)` function.
- Build RNN using `cell` and your `build_rnn(cell, inputs)` function.
- Apply a fully connected layer with a linear activation and `vocab_size` as the number of outputs.

Return the logits and final state in the following tuple (Logits, FinalState) 

In [45]:
def build_nn(cell, rnn_size, input_data, vocab_size, embed_dim, initial_state=None, dtype=tf.float32):
    """
    Build part of the neural network
    :param cell: RNN cell
    :param rnn_size: Size of rnns
    :param input_data: Input data
    :param vocab_size: Vocabulary size
    :param embed_dim: Number of embedding dimensions
    :return: Tuple (Logits, FinalState)
    """
    # DOING: Implement Function
      
    # Apply embeding to the input
    embedded_input = get_embed(input_data, vocab_size, embed_dim)
    print('embedded_input', embedded_input)

    # Build the LSTM cell
    lstm_output, final_state = build_rnn(cell, embedded_input, initial_state, dtype)
    print('lstm_output',lstm_output)
    print('final_state',final_state)


    ## Apply fully connected from lstm_output to vocab_size
    #in_size, out_size = rnn_size, vocab_size
    #logits = build_output(lstm_output, in_size, out_size)
    
    weights = tf.Variable(tf.truncated_normal(shape=[rnn_size, vocab_size], 
                                              mean=0.0, 
                                              stddev=0.1, 
                                              dtype=tf.float32, 
                                              seed=123)) 
    biases = tf.Variable(tf.zeros(shape=[vocab_size], dtype=tf.float32))     
    
    
    def mul_fn(current_input):
        return tf.matmul(current_input, weights) + biases
    logits = tf.map_fn(mul_fn, lstm_output)
    
    # logits = [tf.matmul(output, weights) + biases for output in lstm_output]
    
    return logits, final_state


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_build_nn(build_nn)

embedded_input Tensor("embedding_lookup:0", shape=(128, 5, 300), dtype=float32, device=/device:CPU:0)
cell <tensorflow.contrib.rnn.python.ops.core_rnn_cell_impl.MultiRNNCell object at 0x0000020458EFDC50>
inputs Tensor("embedding_lookup:0", shape=(128, 5, 300), dtype=float32, device=/device:CPU:0)
outputs Tensor("rnn/transpose:0", shape=(128, 5, 256), dtype=float32)
state (LSTMStateTuple(c=<tf.Tensor 'rnn/while/Exit_2:0' shape=(?, 256) dtype=float32>, h=<tf.Tensor 'rnn/while/Exit_3:0' shape=(?, 256) dtype=float32>), LSTMStateTuple(c=<tf.Tensor 'rnn/while/Exit_4:0' shape=(?, 256) dtype=float32>, h=<tf.Tensor 'rnn/while/Exit_5:0' shape=(?, 256) dtype=float32>))
lstm_output Tensor("rnn/transpose:0", shape=(128, 5, 256), dtype=float32)
final_state Tensor("final_state:0", shape=(2, 2, ?, 256), dtype=float32)
Tests Passed


### Batches
Implement `get_batches` to create batches of input and targets using `int_text`.  The batches should be a Numpy array with the shape `(number of batches, 2, batch size, sequence length)`. Each batch contains two elements:
- The first element is a single batch of **input** with the shape `[batch size, sequence length]`
- The second element is a single batch of **targets** with the shape `[batch size, sequence length]`

If you can't fill the last batch with enough data, drop the last batch.

For exmple, `get_batches([1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20], 3, 2)` would return a Numpy array of the following:
```
[
  # First Batch
  [
    # Batch of Input
    [[ 1  2], [ 7  8], [13 14]]
    # Batch of targets
    [[ 2  3], [ 8  9], [14 15]]
  ]

  # Second Batch
  [
    # Batch of Input
    [[ 3  4], [ 9 10], [15 16]]
    # Batch of targets
    [[ 4  5], [10 11], [16 17]]
  ]

  # Third Batch
  [
    # Batch of Input
    [[ 5  6], [11 12], [17 18]]
    # Batch of targets
    [[ 6  7], [12 13], [18  1]]
  ]
]
```

Notice that the last target value in the last batch is the first input value of the first batch. In this case, `1`. This is a common technique used when creating sequence batches, although it is rather unintuitive.

In [46]:
def get_batches(int_text, batch_size, seq_length):
    """
    Return batches of input and target
    :param int_text: Text with the words replaced by their ids
    :param batch_size: The size of batch
    :param seq_length: The length of sequence
    :return: Batches as a Numpy array
    """
    # DONE: Implement Function
    
    # Transform input int_text into an np.array()
    a = np.array(int_text)
    
    # Get the number of words per batch and number of batches we can make
    words_per_batch = batch_size * seq_length
    num_batches = len(a)//words_per_batch

    # Keep only enough words to make full batches
    a = a[:num_batches * words_per_batch]
    
    b = []
    for batch_index in range(num_batches):
        batch_start = batch_index * seq_length
        for input_or_output in range(len(['input', 'output'])):
            input_or_output_start = input_or_output 
            for batch_channel_index in range(batch_size):
                batch_channel_start = batch_start + batch_channel_index * num_batches * seq_length
                for seq_index in range(seq_length):
                    b.append(a[(batch_channel_start + seq_index  + input_or_output_start) % len(a)])  

    b = np.array(b)
    b = b.reshape(-1, 2, batch_size, seq_length)

    return b


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_get_batches(get_batches)

Tests Passed


## Neural Network Training
### Hyperparameters
Tune the following parameters:

- Set `num_epochs` to the number of epochs.
- Set `batch_size` to the batch size.
- Set `rnn_size` to the size of the RNNs.
- Set `embed_dim` to the size of the embedding.
- Set `seq_length` to the length of sequence.
- Set `learning_rate` to the learning rate.
- Set `show_every_n_batches` to the number of batches the neural network should print progress.

In [55]:
# Number of Epochs
num_epochs = 1000
# Batch Size
batch_size = 50
# RNN Size
rnn_size = 128
# Embedding Dimension Size
embed_dim = 64
# Sequence Length
seq_length = 50
# Learning Rate
learning_rate = 2e-3
# Show stats for every n number of batches
show_every_n_batches = 1

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
save_dir = './save'

### Build the Graph
Build the graph using the neural network you implemented.

In [56]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
from tensorflow.contrib import seq2seq

train_graph = tf.Graph()
with train_graph.as_default():
    vocab_size = len(int_to_vocab)
    input_text, targets, lr = get_inputs()
    input_data_shape = tf.shape(input_text)
    cell, initial_state = get_init_cell(input_data_shape[0], rnn_size)
    logits, final_state = build_nn(cell, rnn_size, input_text, vocab_size, embed_dim)

    # Probabilities for generating words
    probs = tf.nn.softmax(logits, name='probs')

    # Loss function
    cost = seq2seq.sequence_loss(
        logits,
        targets,
        tf.ones([input_data_shape[0], input_data_shape[1]]))

    # Optimizer
    optimizer = tf.train.AdamOptimizer(lr)

    # Gradient Clipping
    gradients = optimizer.compute_gradients(cost)
    capped_gradients = [(tf.clip_by_value(grad, -1., 1.), var) for grad, var in gradients if grad is not None]
    train_op = optimizer.apply_gradients(capped_gradients)

embedded_input Tensor("embedding_lookup:0", shape=(?, ?, 64), dtype=float32, device=/device:CPU:0)
cell <tensorflow.contrib.rnn.python.ops.core_rnn_cell_impl.MultiRNNCell object at 0x00000203F2E120B8>
inputs Tensor("embedding_lookup:0", shape=(?, ?, 64), dtype=float32, device=/device:CPU:0)
outputs Tensor("rnn/transpose:0", shape=(?, ?, 128), dtype=float32)
state (LSTMStateTuple(c=<tf.Tensor 'rnn/while/Exit_2:0' shape=(?, 128) dtype=float32>, h=<tf.Tensor 'rnn/while/Exit_3:0' shape=(?, 128) dtype=float32>), LSTMStateTuple(c=<tf.Tensor 'rnn/while/Exit_4:0' shape=(?, 128) dtype=float32>, h=<tf.Tensor 'rnn/while/Exit_5:0' shape=(?, 128) dtype=float32>))
lstm_output Tensor("rnn/transpose:0", shape=(?, ?, 128), dtype=float32)
final_state Tensor("final_state:0", shape=(2, 2, ?, 128), dtype=float32)


## Train
Train the neural network on the preprocessed data.  If you have a hard time getting a good loss, check the [forms](https://discussions.udacity.com/) to see if anyone is having the same problem.

In [57]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
batches = get_batches(int_text, batch_size, seq_length)

with tf.Session(graph=train_graph) as sess:
    sess.run(tf.global_variables_initializer())

    for epoch_i in range(num_epochs):
        state = sess.run(initial_state, {input_text: batches[0][0]})

        for batch_i, (x, y) in enumerate(batches):
            feed = {
                input_text: x,
                targets: y,
                initial_state: state,
                lr: learning_rate}
            train_loss, state, _ = sess.run([cost, final_state, train_op], feed)

            # Show every <show_every_n_batches> batches
            if (epoch_i * len(batches) + batch_i) % show_every_n_batches == 0:
                print('Epoch {:>3} Batch {:>4}/{}   train_loss = {:.3f}'.format(
                    epoch_i,
                    batch_i,
                    len(batches),
                    train_loss))

    # Save Model
    saver = tf.train.Saver()
    saver.save(sess, save_dir)
    print('Model Trained and Saved')

Epoch   0 Batch    0/27   train_loss = 8.821
Epoch   0 Batch    1/27   train_loss = 8.788
Epoch   0 Batch    2/27   train_loss = 8.728
Epoch   0 Batch    3/27   train_loss = 8.587
Epoch   0 Batch    4/27   train_loss = 8.334
Epoch   0 Batch    5/27   train_loss = 8.090
Epoch   0 Batch    6/27   train_loss = 7.871
Epoch   0 Batch    7/27   train_loss = 7.529
Epoch   0 Batch    8/27   train_loss = 7.313
Epoch   0 Batch    9/27   train_loss = 7.087
Epoch   0 Batch   10/27   train_loss = 6.825
Epoch   0 Batch   11/27   train_loss = 6.596
Epoch   0 Batch   12/27   train_loss = 6.630
Epoch   0 Batch   13/27   train_loss = 6.448
Epoch   0 Batch   14/27   train_loss = 6.343
Epoch   0 Batch   15/27   train_loss = 6.394
Epoch   0 Batch   16/27   train_loss = 6.296
Epoch   0 Batch   17/27   train_loss = 6.315
Epoch   0 Batch   18/27   train_loss = 6.182
Epoch   0 Batch   19/27   train_loss = 6.189
Epoch   0 Batch   20/27   train_loss = 6.381
Epoch   0 Batch   21/27   train_loss = 6.280
Epoch   0 

Epoch   6 Batch   21/27   train_loss = 5.642
Epoch   6 Batch   22/27   train_loss = 5.645
Epoch   6 Batch   23/27   train_loss = 5.658
Epoch   6 Batch   24/27   train_loss = 5.668
Epoch   6 Batch   25/27   train_loss = 5.628
Epoch   6 Batch   26/27   train_loss = 5.563
Epoch   7 Batch    0/27   train_loss = 5.718
Epoch   7 Batch    1/27   train_loss = 5.610
Epoch   7 Batch    2/27   train_loss = 5.546
Epoch   7 Batch    3/27   train_loss = 5.654
Epoch   7 Batch    4/27   train_loss = 5.644
Epoch   7 Batch    5/27   train_loss = 5.634
Epoch   7 Batch    6/27   train_loss = 5.696
Epoch   7 Batch    7/27   train_loss = 5.463
Epoch   7 Batch    8/27   train_loss = 5.572
Epoch   7 Batch    9/27   train_loss = 5.578
Epoch   7 Batch   10/27   train_loss = 5.502
Epoch   7 Batch   11/27   train_loss = 5.501
Epoch   7 Batch   12/27   train_loss = 5.658
Epoch   7 Batch   13/27   train_loss = 5.540
Epoch   7 Batch   14/27   train_loss = 5.526
Epoch   7 Batch   15/27   train_loss = 5.663
Epoch   7 

Epoch  13 Batch   15/27   train_loss = 5.206
Epoch  13 Batch   16/27   train_loss = 5.126
Epoch  13 Batch   17/27   train_loss = 5.179
Epoch  13 Batch   18/27   train_loss = 5.085
Epoch  13 Batch   19/27   train_loss = 4.983
Epoch  13 Batch   20/27   train_loss = 5.281
Epoch  13 Batch   21/27   train_loss = 5.089
Epoch  13 Batch   22/27   train_loss = 5.079
Epoch  13 Batch   23/27   train_loss = 5.100
Epoch  13 Batch   24/27   train_loss = 5.114
Epoch  13 Batch   25/27   train_loss = 5.067
Epoch  13 Batch   26/27   train_loss = 5.001
Epoch  14 Batch    0/27   train_loss = 5.181
Epoch  14 Batch    1/27   train_loss = 5.072
Epoch  14 Batch    2/27   train_loss = 4.995
Epoch  14 Batch    3/27   train_loss = 5.089
Epoch  14 Batch    4/27   train_loss = 5.097
Epoch  14 Batch    5/27   train_loss = 5.087
Epoch  14 Batch    6/27   train_loss = 5.143
Epoch  14 Batch    7/27   train_loss = 4.922
Epoch  14 Batch    8/27   train_loss = 5.043
Epoch  14 Batch    9/27   train_loss = 5.027
Epoch  14 

Epoch  20 Batch    9/27   train_loss = 4.717
Epoch  20 Batch   10/27   train_loss = 4.670
Epoch  20 Batch   11/27   train_loss = 4.702
Epoch  20 Batch   12/27   train_loss = 4.839
Epoch  20 Batch   13/27   train_loss = 4.694
Epoch  20 Batch   14/27   train_loss = 4.712
Epoch  20 Batch   15/27   train_loss = 4.854
Epoch  20 Batch   16/27   train_loss = 4.770
Epoch  20 Batch   17/27   train_loss = 4.817
Epoch  20 Batch   18/27   train_loss = 4.743
Epoch  20 Batch   19/27   train_loss = 4.637
Epoch  20 Batch   20/27   train_loss = 4.928
Epoch  20 Batch   21/27   train_loss = 4.728
Epoch  20 Batch   22/27   train_loss = 4.734
Epoch  20 Batch   23/27   train_loss = 4.738
Epoch  20 Batch   24/27   train_loss = 4.767
Epoch  20 Batch   25/27   train_loss = 4.712
Epoch  20 Batch   26/27   train_loss = 4.664
Epoch  21 Batch    0/27   train_loss = 4.822
Epoch  21 Batch    1/27   train_loss = 4.738
Epoch  21 Batch    2/27   train_loss = 4.644
Epoch  21 Batch    3/27   train_loss = 4.746
Epoch  21 

Epoch  27 Batch    3/27   train_loss = 4.558
Epoch  27 Batch    4/27   train_loss = 4.559
Epoch  27 Batch    5/27   train_loss = 4.535
Epoch  27 Batch    6/27   train_loss = 4.595
Epoch  27 Batch    7/27   train_loss = 4.388
Epoch  27 Batch    8/27   train_loss = 4.529
Epoch  27 Batch    9/27   train_loss = 4.483
Epoch  27 Batch   10/27   train_loss = 4.436
Epoch  27 Batch   11/27   train_loss = 4.473
Epoch  27 Batch   12/27   train_loss = 4.615
Epoch  27 Batch   13/27   train_loss = 4.465
Epoch  27 Batch   14/27   train_loss = 4.492
Epoch  27 Batch   15/27   train_loss = 4.622
Epoch  27 Batch   16/27   train_loss = 4.531
Epoch  27 Batch   17/27   train_loss = 4.570
Epoch  27 Batch   18/27   train_loss = 4.515
Epoch  27 Batch   19/27   train_loss = 4.428
Epoch  27 Batch   20/27   train_loss = 4.705
Epoch  27 Batch   21/27   train_loss = 4.498
Epoch  27 Batch   22/27   train_loss = 4.502
Epoch  27 Batch   23/27   train_loss = 4.516
Epoch  27 Batch   24/27   train_loss = 4.558
Epoch  27 

...

Epoch 969 Batch    6/27   train_loss = 0.058
Epoch 969 Batch    7/27   train_loss = 0.059
Epoch 969 Batch    8/27   train_loss = 0.062
Epoch 969 Batch    9/27   train_loss = 0.058
Epoch 969 Batch   10/27   train_loss = 0.059
Epoch 969 Batch   11/27   train_loss = 0.060
Epoch 969 Batch   12/27   train_loss = 0.053
Epoch 969 Batch   13/27   train_loss = 0.056
Epoch 969 Batch   14/27   train_loss = 0.061
Epoch 969 Batch   15/27   train_loss = 0.059
Epoch 969 Batch   16/27   train_loss = 0.056
Epoch 969 Batch   17/27   train_loss = 0.054
Epoch 969 Batch   18/27   train_loss = 0.065
Epoch 969 Batch   19/27   train_loss = 0.058
Epoch 969 Batch   20/27   train_loss = 0.062
Epoch 969 Batch   21/27   train_loss = 0.054
Epoch 969 Batch   22/27   train_loss = 0.054
Epoch 969 Batch   23/27   train_loss = 0.061
Epoch 969 Batch   24/27   train_loss = 0.060
Epoch 969 Batch   25/27   train_loss = 0.061
Epoch 969 Batch   26/27   train_loss = 0.079
Epoch 970 Batch    0/27   train_loss = 0.058
Epoch 970 

Epoch 976 Batch    0/27   train_loss = 0.057
Epoch 976 Batch    1/27   train_loss = 0.060
Epoch 976 Batch    2/27   train_loss = 0.059
Epoch 976 Batch    3/27   train_loss = 0.055
Epoch 976 Batch    4/27   train_loss = 0.060
Epoch 976 Batch    5/27   train_loss = 0.057
Epoch 976 Batch    6/27   train_loss = 0.057
Epoch 976 Batch    7/27   train_loss = 0.057
Epoch 976 Batch    8/27   train_loss = 0.060
Epoch 976 Batch    9/27   train_loss = 0.057
Epoch 976 Batch   10/27   train_loss = 0.057
Epoch 976 Batch   11/27   train_loss = 0.059
Epoch 976 Batch   12/27   train_loss = 0.052
Epoch 976 Batch   13/27   train_loss = 0.055
Epoch 976 Batch   14/27   train_loss = 0.060
Epoch 976 Batch   15/27   train_loss = 0.058
Epoch 976 Batch   16/27   train_loss = 0.055
Epoch 976 Batch   17/27   train_loss = 0.053
Epoch 976 Batch   18/27   train_loss = 0.064
Epoch 976 Batch   19/27   train_loss = 0.057
Epoch 976 Batch   20/27   train_loss = 0.061
Epoch 976 Batch   21/27   train_loss = 0.053
Epoch 976 

Epoch 982 Batch   21/27   train_loss = 0.054
Epoch 982 Batch   22/27   train_loss = 0.053
Epoch 982 Batch   23/27   train_loss = 0.061
Epoch 982 Batch   24/27   train_loss = 0.058
Epoch 982 Batch   25/27   train_loss = 0.059
Epoch 982 Batch   26/27   train_loss = 0.076
Epoch 983 Batch    0/27   train_loss = 0.059
Epoch 983 Batch    1/27   train_loss = 0.059
Epoch 983 Batch    2/27   train_loss = 0.059
Epoch 983 Batch    3/27   train_loss = 0.054
Epoch 983 Batch    4/27   train_loss = 0.061
Epoch 983 Batch    5/27   train_loss = 0.057
Epoch 983 Batch    6/27   train_loss = 0.060
Epoch 983 Batch    7/27   train_loss = 0.059
Epoch 983 Batch    8/27   train_loss = 0.061
Epoch 983 Batch    9/27   train_loss = 0.057
Epoch 983 Batch   10/27   train_loss = 0.059
Epoch 983 Batch   11/27   train_loss = 0.058
Epoch 983 Batch   12/27   train_loss = 0.052
Epoch 983 Batch   13/27   train_loss = 0.055
Epoch 983 Batch   14/27   train_loss = 0.060
Epoch 983 Batch   15/27   train_loss = 0.058
Epoch 983 

Epoch 989 Batch   15/27   train_loss = 0.057
Epoch 989 Batch   16/27   train_loss = 0.054
Epoch 989 Batch   17/27   train_loss = 0.052
Epoch 989 Batch   18/27   train_loss = 0.063
Epoch 989 Batch   19/27   train_loss = 0.056
Epoch 989 Batch   20/27   train_loss = 0.060
Epoch 989 Batch   21/27   train_loss = 0.052
Epoch 989 Batch   22/27   train_loss = 0.052
Epoch 989 Batch   23/27   train_loss = 0.059
Epoch 989 Batch   24/27   train_loss = 0.057
Epoch 989 Batch   25/27   train_loss = 0.058
Epoch 989 Batch   26/27   train_loss = 0.075
Epoch 990 Batch    0/27   train_loss = 0.056
Epoch 990 Batch    1/27   train_loss = 0.058
Epoch 990 Batch    2/27   train_loss = 0.058
Epoch 990 Batch    3/27   train_loss = 0.053
Epoch 990 Batch    4/27   train_loss = 0.059
Epoch 990 Batch    5/27   train_loss = 0.056
Epoch 990 Batch    6/27   train_loss = 0.055
Epoch 990 Batch    7/27   train_loss = 0.056
Epoch 990 Batch    8/27   train_loss = 0.059
Epoch 990 Batch    9/27   train_loss = 0.055
Epoch 990 

Epoch 996 Batch    9/27   train_loss = 0.055
Epoch 996 Batch   10/27   train_loss = 0.055
Epoch 996 Batch   11/27   train_loss = 0.057
Epoch 996 Batch   12/27   train_loss = 0.050
Epoch 996 Batch   13/27   train_loss = 0.053
Epoch 996 Batch   14/27   train_loss = 0.058
Epoch 996 Batch   15/27   train_loss = 0.056
Epoch 996 Batch   16/27   train_loss = 0.053
Epoch 996 Batch   17/27   train_loss = 0.051
Epoch 996 Batch   18/27   train_loss = 0.062
Epoch 996 Batch   19/27   train_loss = 0.055
Epoch 996 Batch   20/27   train_loss = 0.059
Epoch 996 Batch   21/27   train_loss = 0.051
Epoch 996 Batch   22/27   train_loss = 0.051
Epoch 996 Batch   23/27   train_loss = 0.058
Epoch 996 Batch   24/27   train_loss = 0.057
Epoch 996 Batch   25/27   train_loss = 0.057
Epoch 996 Batch   26/27   train_loss = 0.074
Epoch 997 Batch    0/27   train_loss = 0.055
Epoch 997 Batch    1/27   train_loss = 0.057
Epoch 997 Batch    2/27   train_loss = 0.057
Epoch 997 Batch    3/27   train_loss = 0.052
Epoch 997 

## Save Parameters
Save `seq_length` and `save_dir` for generating a new TV script.

In [58]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
# Save parameters for checkpoint
helper.save_params((seq_length, save_dir))

# Checkpoint

In [59]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import tensorflow as tf
import numpy as np
import helper
import problem_unittests as tests

_, vocab_to_int, int_to_vocab, token_dict = helper.load_preprocess()
seq_length, load_dir = helper.load_params()

## Implement Generate Functions
### Get Tensors
Get tensors from `loaded_graph` using the function [`get_tensor_by_name()`](https://www.tensorflow.org/api_docs/python/tf/Graph#get_tensor_by_name).  Get the tensors using the following names:
- "input:0"
- "initial_state:0"
- "final_state:0"
- "probs:0"

Return the tensors in the following tuple `(InputTensor, InitialStateTensor, FinalStateTensor, ProbsTensor)` 

In [60]:
def get_tensors(loaded_graph):
    """
    Get input, initial state, final state, and probabilities tensor from <loaded_graph>
    :param loaded_graph: TensorFlow graph loaded from file
    :return: Tuple (InputTensor, InitialStateTensor, FinalStateTensor, ProbsTensor)
    """
    # DONE: Implement Function
    InputTensor = loaded_graph.get_tensor_by_name('input:0')
    InitialStateTensor = loaded_graph.get_tensor_by_name('initial_state:0')
    FinalStateTensor = loaded_graph.get_tensor_by_name('final_state:0')
    ProbsTensor = loaded_graph.get_tensor_by_name('probs:0')
    return InputTensor, InitialStateTensor, FinalStateTensor, ProbsTensor


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_get_tensors(get_tensors)

Tests Passed


### Choose Word
Implement the `pick_word()` function to select the next word using `probabilities`.

In [61]:
def pick_word(probabilities, int_to_vocab, top_n = 5):
    """
    Pick the next word in the generated text
    :param probabilities: Probabilites of the next word
    :param int_to_vocab: Dictionary of word ids as the keys and words as the values
    :return: String of the predicted word
    """
    # DONE: Implement Function
    p = probabilities
    p[np.argsort(p)[:-top_n]] = 0
    p = p / np.sum(p)
    w = np.random.choice(np.array(list(int_to_vocab.values())), 1, p=p)[0]
    
    return w

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_pick_word(pick_word)

Tests Passed


## Generate TV Script
This will generate the TV script for you.  Set `gen_length` to the length of TV script you want to generate.

In [64]:
gen_length = 200
# homer_simpson, moe_szyslak, or Barney_Gumble
prime_word = 'moe_szyslak'

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
loaded_graph = tf.Graph()
with tf.Session(graph=loaded_graph) as sess:
    # Load saved model
    loader = tf.train.import_meta_graph(load_dir + '.meta')
    loader.restore(sess, load_dir)

    # Get Tensors from loaded model
    input_text, initial_state, final_state, probs = get_tensors(loaded_graph)

    # Sentences generation setup
    gen_sentences = [prime_word + ':']
    prev_state = sess.run(initial_state, {input_text: np.array([[1]])})

    # Generate sentences
    for n in range(gen_length):
        # Dynamic Input
        dyn_input = [[vocab_to_int[word] for word in gen_sentences[-seq_length:]]]
        dyn_seq_length = len(dyn_input[0])

        # Get Prediction
        probabilities, prev_state = sess.run(
            [probs, final_state],
            {input_text: dyn_input, initial_state: prev_state})
        
        pred_word = pick_word(probabilities[dyn_seq_length-1], int_to_vocab)

        gen_sentences.append(pred_word)
    
    # Remove tokens
    tv_script = ' '.join(gen_sentences)
    for key, token in token_dict.items():
        ending = ' ' if key in ['\n', '(', '"'] else ''
        tv_script = tv_script.replace(' ' + token.lower(), key)
    tv_script = tv_script.replace('\n ', '\n')
    tv_script = tv_script.replace('( ', '(')
        
    print(tv_script)

moe_szyslak: well, if you change your mind, you know where i am.
patty_bouvier: in my nightmares.
moe_szyslak: i'm gonna stop now.


homer_simpson: two glasses of wine, moe.
moe_szyslak: wine? jeez. no one ever orders that.
moe_szyslak: homer, ah, you care for a tropical was in till mr." i hate my old hand me why.(to moe)... so well... who the hell's been putting beers on his tab?
barney_gumble:(tentative annoyed grunt)(then) woo hoo?(then, searching) oh, i don't want you to meet my new special lady. say hello to starla.
starla: can i have the keys to the car.
carl_carlson: happy on tv.
barney_gumble: whoa!
homer_simpson: i... well, i wish mr. now, do that you girls comes down to a man.(to moe) usually you do me to cut to my carl.
moe_szyslak: oh.
homer_simpson: yeah,


# The TV Script is Nonsensical
It's ok if the TV script doesn't make any sense.  We trained on less than a megabyte of text.  In order to get good results, you'll have to use a smaller vocabulary or get more data.  Luckly there's more data!  As we mentioned in the begging of this project, this is a subset of [another dataset](https://www.kaggle.com/wcukierski/the-simpsons-by-the-data).  We didn't have you train on all the data, because that would take too long.  However, you are free to train your neural network on all the data.  After you complete the project, of course.
# Submitting This Project
When submitting this project, make sure to run all the cells before saving the notebook. Save the notebook file as "dlnd_tv_script_generation.ipynb" and save it as a HTML file under "File" -> "Download as". Include the "helper.py" and "problem_unittests.py" files in your submission.